In [1]:
import os

import numpy as np
import pandas as pd

import spacy

from sklearn.model_selection import train_test_split

In [2]:
RANDOM_STATE = 42
DATA_FOLDER = "data/"

#input data files
TRAINING_DATA_FILE_X = "training_set_clean_only_text.txt"
TRAINING_DATA_FILE_y = "training_set_clean_only_tags.txt"


# output data files (after preprosessing and splitting)
X_TRAIN_FILE_NAME = "X_train.csv"
X_VALID_FILE_NAME = "X_valid.csv"

Y_TRAIN_FILE_NAME = "y_train.csv"
Y_VALID_FILE_NAME = "y_valid.csv"

In [3]:
with open(os.path.join(DATA_FOLDER, TRAINING_DATA_FILE_X), "r") as input_file:
    X = input_file.readlines()
    
with open(os.path.join(DATA_FOLDER, TRAINING_DATA_FILE_y), "r") as input_file:
    y = input_file.readlines()
    
    
X = pd.DataFrame([x.strip() for x in X], columns=["text"])
y = pd.DataFrame([x.strip() for x in y], columns=["label"])

In [4]:
X.head()

,text
0,Dla mnie faworytem do tytułu będzie Cracovia. ...
1,@anonymized_account @anonymized_account Brawo ...
2,"@anonymized_account @anonymized_account Super,..."
3,@anonymized_account @anonymized_account Musi. ...
4,"Odrzut natychmiastowy, kwaśna mina, mam problem"


In [5]:
y.head()

,label
0,0
1,0
2,0
3,0
4,0


In [6]:
X.shape, y.shape

((10041, 1), (10041, 1))

Rzut okiem na przykłądowy pełny tekst:

In [7]:
X.iloc[0][0]

'Dla mnie faworytem do tytułu będzie Cracovia. Zobaczymy, czy typ się sprawdzi.'

Czy nie ma jakichś błędów w danych?

In [8]:
pd.isnull(X.text).sum(), pd.isna(X.text).sum()

(0, 0)

Czy są puste napisy?

In [9]:
text_lengths = X.text.str.len()
text_lengths.min(), text_lengths.max()

(6, 214)

Rzut okiem na podejrzanie krótkie napisy:

In [10]:
X[text_lengths == 6]

,text
528,#NAME?
1918,#NAME?
2081,#NAME?
3201,#NAME?
5567,#NAME?
5735,#NAME?
6085,#NAME?
7341,#NAME?
7424,#NAME?
8746,#NAME?


Z opisu danych (a konkretnei fragmentu 'The preprocessing, if used, will be applied mostly for cases when information about a private person is revealed to the public") wnisokuję, że to anonimizacja danych osobowych (imion czy nazwisk).

Sprawdzam, czy może gdzieś jeszcze się to pojawia?

In [11]:
X.text[X.text.apply(lambda x: "#NAME?" in x)]

528     #NAME?
1918    #NAME?
2081    #NAME?
3201    #NAME?
5567    #NAME?
5735    #NAME?
6085    #NAME?
7341    #NAME?
7424    #NAME?
8746    #NAME?
8811    #NAME?
8857    #NAME?
9809    #NAME?
Name: text, dtype: object

In [12]:
y[X.text.apply(lambda x: "#NAME?" in x)]

,label
528,0
1918,0
2081,0
3201,0
5567,0
5735,0
6085,0
7341,0
7424,0
8746,0


Same zera. 

Usunę te obserwacje, ponieważ wprowadzają one tylko szum do danych i nic nie wnoszą. Jeżeli w danych, an których będziemy dokonywać predykcji pojawi się tweet takiej postacji, to można byłoby przyjąć regułkę, że przypisujemy tam zero. Natomiast pominiemy to, ponieważ zakłądam, że w realnym scenariuszu biznesowym takich tweetów nie będzie, bo to nie ma sensu.

In [13]:
rows_to_drop = X.text.apply(lambda x: "#NAME?" in x)

X = X[~rows_to_drop]
y = y[~rows_to_drop]

X.shape[0], y.shape[0]

(10028, 10028)

Rzut okiem na pozostałe "skrajne" tweety (drugie od najkrótszych i najdłuższe) - czy nie ma tam jakichś problemów.

In [14]:
text_lengths = X.text.str.len()

In [15]:
text_lengths.min(), text_lengths.max()

(18, 214)

In [16]:
X[text_lengths == text_lengths.min()].iloc[0,0]

'Co to miało być? 😂'

In [17]:
X[text_lengths == text_lengths.max()].iloc[0,0]

'@anonymized_account @anonymized_account @anonymized_account @anonymized_account @anonymized_account @anonymized_account @anonymized_account @anonymized_account @anonymized_account Dla nich każda prawda to chamstwo.'

To pobieżne sprawdzenie, ale ze względu na czas uznaję, że dane już są "czyste". W praktyce, sprawdzałbym to dokładniej i dłużej.

Rzut okiem na rozkład zmiennej celu.

In [18]:
y.value_counts(normalize=True)

label
0        0.915138
2        0.059633
1        0.025229
dtype: float64

Rozkład silnie niezbalansowany - potrzebna dopasowana miara jakości (c zresztą wiadomo już z opisu danych na polevalu, a i tak nawet bez tego to było do przewidzenia:)) oraz prawdopodobnie przydatne techniki dla niezbalansowanych danych.

Dodamy do danych text w wersji zlematyzowanej.

In [19]:
NLP = spacy.load('pl_core_news_sm')

In [20]:
def lemmatize(x):
    doc = NLP(x) 
    return " ".join([token.lemma_ for token in doc])

In [21]:
X["text_lemmatized"] = X.text.apply(lemmatize)

Rzut okiem czy się coś nie zepsuło:

In [22]:
X

,text,text_lemmatized
0,Dla mnie faworytem do tytułu będzie Cracovia. ...,dla ja faworyt do tytuł być cracovia . zobaczy...
1,@anonymized_account @anonymized_account Brawo ...,@anonymized_account @anonymized_account brawo ...
2,"@anonymized_account @anonymized_account Super,...",@anonymized_account @anonymized_account super ...
3,@anonymized_account @anonymized_account Musi. ...,@anonymized_account @anonymized_account musić ...
4,"Odrzut natychmiastowy, kwaśna mina, mam problem","odrzut natychmiastowy , kwaśna mina , mieć pro..."
...,...,...
10036,@anonymized_account Ty zagrasz? Nie wiedziałem 😉,@anonymized_account ty zagrać ? nie wiedzieć 😉
10037,@anonymized_account @anonymized_account A VAR ...,@anonymized_account @anonymized_account a var ...
10038,@anonymized_account @anonymized_account Szanow...,@anonymized_account @anonymized_account szanow...
10039,@anonymized_account @anonymized_account @anony...,@anonymized_account @anonymized_account @anony...


Podział na dane treningowe i walidacyjne wykonuję z zachowaniem proporcji klas w obu podgrupach. Niektórzy uznają to za podejście niepoprawne (ponieważ wartości z danych wpływają na podział), ale ja uważam, że tak należy robić, ponieważ w przeciwnym przypadku pechowego podziału, ocena może być mocno niestabilna/zaburzona. Koniec konieców, i tak to jest tylko zbiór walidacyjny, który zostanie wykorzystany do optymalizacji rozwiązań - ostateczna ocena zostanie jeszcze przeprowadzona na zbiorze testowym dostarczonym w konkursie.

In [23]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=1000, stratify=y, random_state=RANDOM_STATE)

In [24]:
y_train.value_counts(normalize=True), y_valid.value_counts(normalize=True)

(label
 0        0.915153
 2        0.059592
 1        0.025255
 dtype: float64,
 label
 0        0.915
 2        0.060
 1        0.025
 dtype: float64)

In [25]:
X_train.to_csv(os.path.join(DATA_FOLDER, X_TRAIN_FILE_NAME))
X_valid.to_csv(os.path.join(DATA_FOLDER, X_VALID_FILE_NAME))

y_train.to_csv(os.path.join(DATA_FOLDER, Y_TRAIN_FILE_NAME))
y_valid.to_csv(os.path.join(DATA_FOLDER, Y_VALID_FILE_NAME))